### Extract one month data for new medicines since we need to predict for only 1 month

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import json
import math
import os
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error

In [3]:
path=os.getcwd()
path

'C:\\Users\\jayam\\Downloads\\7879phd'

In [12]:
test=pd.read_csv(path +"\\test_final.csv", sep=",", na_values=["?",",","#","NaN","unknown",""],usecols=["date","id","city_medicine","old_new","city","medicine"])# read the test data

In [3]:
train=pd.read_csv(path +"\\train_final.csv", sep=",", na_values=["?",",","#","NaN","unknown",""],usecols=["date","city_medicine","sales","old_new"])# read the train data

In [5]:
train["date"]=pd.to_datetime(train["date"])#convert to datetime

In [6]:
train.head(2)# see head

,date,sales,city_medicine,old_new
0,2015-01-02,24.0,1_1,old
1,2015-01-02,144.0,1_2,old


In [7]:
newtrn_city_meds=list(train[train["old_new"]=="new"]["city_medicine"].unique())# collect all the new city_medicine combinations in train

In [13]:
newtst_city_meds=list(test[test["old_new"]=="new"]["city_medicine"].unique())# collect all the new city_medicine combinations in test

In [23]:
len(newtrn_city_meds),len(newtst_city_meds)## getting length of new medicine list for train and test data

(20720, 313)

- So there are 313 new city medicines in test data which are not present in train data, so how do we predict the sales of unknown medicines?

- As we need to predict the sales of unknown medicines also for only 1 month ,let us extract the sales of all new medicines in train for only 1 month

In [8]:
newtrn_city_meds# list of new city medicinesin train

['1_1205',
 '1_1207',
 '1_1209',
 '1_1214',
 '1_1215',
 '1_1218',
 '1_1219',
 '1_1220',
 '1_1222',
 '1_1223',
 '1_1226',
 '1_1230',
 '1_1235',
 '1_1237',
 '1_1239',
 '1_1240',
 '1_1243',
 '1_1244',
 '1_1245',
 '1_1246',
 '1_1247',
 '1_1252',
 '1_1254',
 '1_1260',
 '1_1261',
 '1_1264',
 '1_1265',
 '1_1270',
 '1_1276',
 '1_1278',
 '1_1280',
 '1_1281',
 '1_1288',
 '1_1289',
 '1_1295',
 '1_1296',
 '1_1302',
 '1_1318',
 '1_1322',
 '1_1327',
 '1_1332',
 '1_1335',
 '1_1340',
 '1_1343',
 '1_1344',
 '1_1348',
 '1_1349',
 '1_1366',
 '1_1368',
 '1_1371',
 '1_1375',
 '1_1386',
 '2_8',
 '2_15',
 '2_36',
 '2_43',
 '2_45',
 '2_77',
 '2_95',
 '2_97',
 '2_107',
 '2_109',
 '2_135',
 '2_144',
 '2_168',
 '2_187',
 '2_197',
 '2_217',
 '2_302',
 '2_303',
 '2_329',
 '2_336',
 '2_337',
 '2_353',
 '2_360',
 '2_375',
 '2_400',
 '2_415',
 '2_444',
 '2_448',
 '2_467',
 '2_498',
 '2_516',
 '2_609',
 '2_618',
 '2_654',
 '2_657',
 '2_778',
 '2_784',
 '2_799',
 '2_801',
 '2_851',
 '2_882',
 '2_897',
 '2_920',
 '2_941

- Let us randomly check how the sales are for a medicine

In [14]:
 df1=train[train["city_medicine"]=="1_1205"].reset_index(drop=True).drop(["old_new"],axis=1)

In [15]:
df1

,date,sales,city_medicine
0,2015-01-03,20.0,1_1205
1,2015-01-07,16.0,1_1205
2,2015-01-12,12.0,1_1205
3,2015-01-14,4.0,1_1205
4,2015-01-16,28.0,1_1205
...,...,...,...
936,2018-06-25,16.0,1_1205
937,2018-06-26,16.0,1_1205
938,2018-06-27,8.0,1_1205
939,2018-06-28,36.0,1_1205


- We could see there are some missing dates in between where there are no sales
- let us create a function for imputing dates and add sales as zero
- Then extract the first 30 days sales for all the new medicines in each city from train data

In [14]:
df1[df1.index==0]["date"][0]# extracting first day

Timestamp('2015-01-03 00:00:00')

In [37]:
def get_30_rows(city_meds):# function for extracting sales of all the new medicines
    fin=pd.DataFrame()
    for med in city_meds:# for each medicine in train data
        df=train[train["city_medicine"]==med].reset_index(drop=True).drop(["old_new"],axis=1)# take subset 
        start_date=df[df.index==0]["date"][0]# extract  the first day date
        end_date = pd.to_datetime(start_date) + pd.DateOffset(days=31)# add 31 days to start date
        dates=pd.DataFrame()# create dates dataframe for 30 days dates
        dates["date"]=pd.date_range(start=start_date,end=pd.to_datetime(start_date) + pd.DateOffset(days=31),freq="d")
        dates["city_medicine"]=med # creating a column in dates dataframe for city_medicine
        cond = df.date <= end_date # creating condition for extracting the sales till 30 days for each medicine
        df=df[cond]# pass the condition inside the subset dataframe and merge with dates dataframe
        df=dates.merge(df,on=["date","city_medicine"],how="left").fillna(0).reset_index().rename(columns={"index":"ID"})
        fin=pd.concat([fin,df])# we have created id column to indicate the first to last 30 day sales for each medicine
    return fin    # so, now we have an index as reference for each day for each medicine 

In [38]:
new_meds_df=get_30_rows(newtrn_city_meds)# run the function get_30_rows and get all the sales data for 30 days

In [39]:
new_meds_df.to_csv("new_meds_ver1.csv")# save it for future reference

- so the next strategy is to take the average sales of all the new medicines in each city for each day 

In [43]:
new_meds_df[["city","medicine"]]=new_meds_df["city_medicine"].str.split("_",expand=True)# split city medicine

In [54]:
grp_city_sales=new_meds_df.groupby(["city","ID"])["sales"].mean().reset_index()# groupby city and ID and take mean of sales

In [58]:
grp_city_sales.columns# list of columns after groupby data frame

Index(['city', 'ID', 'sales'], dtype='object')

In [61]:
grp_city_sales["city"]=grp_city_sales["city"].astype("int")# convert city type to int

In [75]:
grp_city_sales.to_csv("new_cityavg_sales.csv",index=False)# save the average sales dataframe for future reference

In [96]:
newtst_city_meds # list of new test medicines

['1_3389',
 '1_3390',
 '2_2654',
 '3_3388',
 '3_3389',
 '4_3389',
 '4_3390',
 '5_3386',
 '5_3363',
 '5_3390',
 '6_3389',
 '7_3388',
 '9_3388',
 '9_3384',
 '10_3371',
 '10_3380',
 '1_3387',
 '2_3387',
 '3_3379',
 '3_3390',
 '4_3391',
 '5_3391',
 '6_3392',
 '6_3390',
 '7_3368',
 '7_3391',
 '7_3392',
 '8_3390',
 '8_3393',
 '9_3390',
 '9_3393',
 '10_3388',
 '10_3378',
 '10_3370',
 '10_3389',
 '10_3390',
 '1_3393',
 '2_3392',
 '3_3392',
 '4_3392',
 '7_3389',
 '7_3390',
 '8_3391',
 '10_3324',
 '2_3368',
 '2_3390',
 '3_3393',
 '5_3392',
 '5_3393',
 '6_3391',
 '7_3366',
 '7_3393',
 '9_3391',
 '1_3392',
 '2_3391',
 '4_3382',
 '5_3216',
 '6_3380',
 '6_3394',
 '10_3393',
 '4_3383',
 '4_3394',
 '5_3345',
 '7_3394',
 '8_3385',
 '9_3350',
 '9_3392',
 '1_3394',
 '3_3380',
 '7_3382',
 '8_3363',
 '9_3394',
 '1_3378',
 '1_3395',
 '3_3395',
 '4_3395',
 '6_3393',
 '7_3383',
 '7_3395',
 '8_2868',
 '8_3381',
 '8_3394',
 '8_3395',
 '9_3378',
 '2_3393',
 '4_3393',
 '5_2861',
 '5_3395',
 '3_3391',
 '3_3394',
 

In [112]:
test[test["city_medicine"].isin(newtst_city_meds)]# so there are 3196 rows of new medicines in test data

,id,city,medicine,date,city_medicine,old_new
2486,2487,1,3389,2018-07-01,1_3389,new
2487,2488,1,3390,2018-07-01,1_3390,new
4104,4105,2,2654,2018-07-01,2_2654,new
7281,7282,3,3388,2018-07-01,3_3388,new
7305,7306,3,3389,2018-07-01,3_3389,new
...,...,...,...,...,...,...
773448,773449,10,3408,2018-07-31,10_3408,old
773449,773450,10,3421,2018-07-31,10_3421,new
773450,773451,10,3418,2018-07-31,10_3418,new
773451,773452,10,3410,2018-07-31,10_3410,new


In [105]:
test[test["city_medicine"]=="1_3396"]# let us check how the sales are in test data for a medicine 

,id,city,medicine,date,city_medicine,old_new
303966,303967,1,3396,2018-07-13,1_3396,new
328057,328058,1,3396,2018-07-14,1_3396,old
351413,351414,1,3396,2018-07-15,1_3396,old
376201,376202,1,3396,2018-07-16,1_3396,old
402388,402389,1,3396,2018-07-17,1_3396,old
428486,428487,1,3396,2018-07-18,1_3396,old
453010,453011,1,3396,2018-07-19,1_3396,old
477119,477120,1,3396,2018-07-20,1_3396,old
501422,501423,1,3396,2018-07-21,1_3396,old
525275,525276,1,3396,2018-07-22,1_3396,old


- So we should impute missing dates and  predict the sales
- How do we predict the sales for new medicines in test data ?
- Simple way is the average sales of all the new medicines taken on that day in train data for any new medicine in test data 

In [111]:
def new_city_med_sales(lst): # same function as in train data for imputing dates  same explanation follows with little changes
    fin=pd.DataFrame()
    for med in lst:
        df=test[test["city_medicine"]==med].reset_index(drop=True)
        df["date"]=pd.to_datetime(df["date"])
        start_date=df[df.index==0]["date"][0]
        end_date = pd.to_datetime(start_date) + pd.DateOffset(days=31)
        dates=pd.DataFrame()
        dates["date"]=pd.date_range(start=start_date,end=pd.to_datetime(start_date) + pd.DateOffset(days=31),freq="d")
        dates["city_medicine"]=med
        dates["city"]=df[df.index==0]["city"][0]# adding city and medicine column to avoid splitting again
        dates["medicine"]=df[df.index==0]["medicine"][0]
        dates=dates.merge(df,on=["city_medicine","date","city","medicine"],how="left").fillna(0).reset_index().rename(columns={"index":"ID"})
        fin=pd.concat([fin,dates])
    return fin

In [97]:
len(newtst_city_meds)

313

In [100]:
newtst_city_meds

['1_3389',
 '1_3390',
 '2_2654',
 '3_3388',
 '3_3389',
 '4_3389',
 '4_3390',
 '5_3386',
 '5_3363',
 '5_3390',
 '6_3389',
 '7_3388',
 '9_3388',
 '9_3384',
 '10_3371',
 '10_3380',
 '1_3387',
 '2_3387',
 '3_3379',
 '3_3390',
 '4_3391',
 '5_3391',
 '6_3392',
 '6_3390',
 '7_3368',
 '7_3391',
 '7_3392',
 '8_3390',
 '8_3393',
 '9_3390',
 '9_3393',
 '10_3388',
 '10_3378',
 '10_3370',
 '10_3389',
 '10_3390',
 '1_3393',
 '2_3392',
 '3_3392',
 '4_3392',
 '7_3389',
 '7_3390',
 '8_3391',
 '10_3324',
 '2_3368',
 '2_3390',
 '3_3393',
 '5_3392',
 '5_3393',
 '6_3391',
 '7_3366',
 '7_3393',
 '9_3391',
 '1_3392',
 '2_3391',
 '4_3382',
 '5_3216',
 '6_3380',
 '6_3394',
 '10_3393',
 '4_3383',
 '4_3394',
 '5_3345',
 '7_3394',
 '8_3385',
 '9_3350',
 '9_3392',
 '1_3394',
 '3_3380',
 '7_3382',
 '8_3363',
 '9_3394',
 '1_3378',
 '1_3395',
 '3_3395',
 '4_3395',
 '6_3393',
 '7_3383',
 '7_3395',
 '8_2868',
 '8_3381',
 '8_3394',
 '8_3395',
 '9_3378',
 '2_3393',
 '4_3393',
 '5_2861',
 '5_3395',
 '3_3391',
 '3_3394',
 

In [114]:
new_tst_city_med_df=new_city_med_sales(newtst_city_meds)# running the function

In [125]:
new_tst_city_med_df

,ID,date,city_medicine,city,medicine,id,old_new,sales
0,0,2018-07-01,1_3389,1,3389,2487.0,new,55.458491
1,1,2018-07-02,1_3389,1,3389,28034.0,old,52.035753
2,2,2018-07-03,1_3389,1,3389,54478.0,old,57.219777
3,3,2018-07-04,1_3389,1,3389,80869.0,old,59.004786
4,4,2018-07-05,1_3389,1,3389,105807.0,old,44.757251
...,...,...,...,...,...,...,...,...
10011,27,2018-08-27,10_3410,10,3410,0.0,0,22.314706
10012,28,2018-08-28,10_3410,10,3410,0.0,0,22.168467
10013,29,2018-08-29,10_3410,10,3410,0.0,0,22.863717
10014,30,2018-08-30,10_3410,10,3410,0.0,0,16.344901


- After imputing the dates merge it with avg sales data frame of train data on city and ID to get sales

In [117]:
new_tst_city_med_df=new_tst_city_med_df.merge(new_ctymeds_df,on=["city","ID"],how="left")

In [120]:
final_preds=new_tst_city_med_df[new_tst_city_med_df["id"]!=0]# to filter only required test data w.r.t rows we need to predict

In [122]:
final_preds["sales"].value_counts().values

array([37, 35, 34, 32, 31, 31, 31, 30, 30, 29, 27, 26, 25, 25, 25, 24, 24,
       24, 23, 23, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 19, 19, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13,
       13, 13, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  5,  5,  5

In [123]:
final_preds.to_csv("new_citymed_avgpredictions.csv",index=False)# saving for future reference

In [46]:
final_preds["sales"].value_counts()

66.087849    37
61.481200    35
57.710925    35
33.926537    34
59.487500    32
             ..
42.914604     1
36.991692     1
27.287701     1
59.424253     1
45.465638     1
Name: sales, Length: 282, dtype: int64

#### End of exercise